In [2]:
num = 12
num

12

In [1]:
print("heeyyy")

heeyyy


In [2]:
!pip install openai

In [6]:
!pip install python-dotenv


In [1]:
import os
from dotenv import load_dotenv
from openai import OpenAI

# Load from .env file
load_dotenv()
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

response = client.responses.create(
    model="gpt-4o-mini",
    input="hey"
)

print(response.output_text)


Hey! How can I assist you today?


In [3]:
!pip install langchain_openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 8.6 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 759.5/759.5 kB 10.5 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 801.9/801.9 kB 10.7 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 11.3 MB/s  0:00:00 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15/15 [langchain_openai][langchain-core]


In [11]:
!pip show langchain-openai


Name: langchain-openai
Version: 0.3.32
Summary: An integration package connecting OpenAI and LangChain
Home-page: 
Author: 
Author-email: 
License: MIT
Location: /home/Musaed/Documents/GitHub_My-Repos/Task1/.venv/lib/python3.13/site-packages
Requires: langchain-core, openai, tiktoken
Required-by: 


## Testing The OpenAI API

In [7]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI

# Load from .env file
load_dotenv()

# Initialize LangChain OpenAI wrapper
llm = ChatOpenAI(
    model="gpt-4o-mini",
    api_key=os.getenv("OPENAI_API_KEY")
)

# Simple call
response = llm.invoke("hey")

print(response.content)


Hello! How can I assist you today?


In [ ]:
import os
from dotenv import load_dotenv
from openai import OpenAI

load_dotenv()
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# ---- Strategy A prompts ----
SYSTEM = (
    "You are a helpful Lucidya product assistant. Detect the user’s language and respond "
    "in the same language (Arabic ↔ English). Prioritize correctness, simplicity, and actionable steps. "
    "If unsure, ask one clarifying question before answering. Never invent features; if information is missing, "
    "say so and suggest what to check."
)

DEV = (
    "Assistant Instruction:\n"
    "Use short sentences. Prefer lists. Provide at most one example. "
    "End with a brief next step. If Arabic, write in Modern Standard Arabic."
)

def generate_response(end_user_query: str, kb_chunks: str | None = None) -> str:
    messages = [
        {"role": "system", "content": SYSTEM},
        {"role": "system", "content": DEV},
    ]
    if kb_chunks and kb_chunks.strip():
        messages.append({"role": "system", "content": "[Lucidya KB snippets]\n" + kb_chunks})

    messages.append({"role": "user", "content": end_user_query})

    resp = client.chat.completions.create(
        model=os.getenv("OPENAI_MODEL", "gpt-4o-mini"),
        temperature=float(os.getenv("OPENAI_TEMPERATURE", "0.2")),
        messages=messages,
    )
    return resp.choices[0].message.content

# ---- Example ----
if __name__ == "__main__":
    print(generate_response("How do I connect Twitter to Lucidya?"))
    # kb = "- Twitter integration: Settings > Channels > Add Twitter.\n- Requires app keys and callback URL.\n- Sync delay: ~5 min."
    # print(generate_response("كيف أربط تويتر بحسابي؟", kb))


To connect Twitter to Lucidya, follow these steps:

1. **Log in to Lucidya**.
2. **Go to Settings**.
3. **Select Integrations**.
4. **Find Twitter**.
5. **Click on Connect**.
6. **Authorize Lucidya** to access your Twitter account.

If you encounter any issues, check your Twitter account permissions. 

Next, try connecting and let me know if you need further help!


In [18]:
import os, time, json
from dotenv import load_dotenv
from openai import OpenAI

load_dotenv()
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# ---- Strategy A prompts ----
SYSTEM = (
    "You are a helpful Lucidya product assistant. Detect the user’s language and respond "
    "in the same language (Arabic ↔ English). Prioritize correctness, simplicity, and actionable steps. "
    "If unsure, ask one clarifying question before answering. Never invent features; if information is missing, "
    "say so and suggest what to check."
)

DEV = (
    "Assistant Instruction:\n"
    "Use short sentences. Prefer lists. Provide at most one example. "
    "End with a brief next step. If Arabic, write in Modern Standard Arabic."
)

def generate_response(end_user_query: str, kb_chunks: str | None = None) -> str:
    messages = [
        {"role": "system", "content": SYSTEM},
        {"role": "system", "content": DEV},
    ]
    if kb_chunks and kb_chunks.strip():
        messages.append({"role": "system", "content": "[Lucidya KB snippets]\n" + kb_chunks})

    messages.append({"role": "user", "content": end_user_query})

    # Log the full prompt
    print("----- Prompt sent to model -----")
    print(json.dumps(messages, indent=2, ensure_ascii=False))

    start = time.time()
    resp = client.chat.completions.create(
        model=os.getenv("OPENAI_MODEL", "gpt-4o-mini"),
        temperature=float(os.getenv("OPENAI_TEMPERATURE", "0.2")),
        messages=messages,
    )
    elapsed = time.time() - start

    text = resp.choices[0].message.content
    usage = resp.usage

    print("\n----- Response -----")
    print(text)
    print("\n----- Trace Info -----")
    print("ID:", resp.id)
    print("Model:", resp.model)
    print("Prompt tokens:", usage.prompt_tokens)
    print("Completion tokens:", usage.completion_tokens)
    print("Total tokens:", usage.total_tokens)
    print("Elapsed time (s):", round(elapsed, 3))

    return text  # you can use it programmatically if needed

# ---- Example ----
if __name__ == "__main__":

    generate_response("How do I connect Twitter to Lucidya?")



----- Prompt sent to model -----
[
  {
    "role": "system",
    "content": "You are a helpful Lucidya product assistant. Detect the user’s language and respond in the same language (Arabic ↔ English). Prioritize correctness, simplicity, and actionable steps. If unsure, ask one clarifying question before answering. Never invent features; if information is missing, say so and suggest what to check.Assistant Instruction:\nUse short sentences. Prefer lists. Provide at most one example. End with a brief next step. If Arabic, write in Modern Standard Arabic."
  },
  {
    "role": "user",
    "content": "How do I connect Twitter to Lucidya?"
  }
]

----- Response -----
To connect Twitter to Lucidya, follow these steps:

1. Log in to your Lucidya account.
2. Go to the "Integrations" section.
3. Find Twitter in the list of available integrations.
4. Click on "Connect" or "Add".
5. Authorize Lucidya to access your Twitter account.
6. Follow any additional prompts to complete the setup.

Next, c

----- Prompt sent to model -----
[
  {
    "role": "system",
    "content": "You are a helpful Lucidya product assistant. Detect the user’s language and respond in the same language (Arabic ↔ English). Prioritize correctness, simplicity, and actionable steps. If unsure, ask one clarifying question before answering. Never invent features; if information is missing, say so and suggest what to check."
  },
  {
    "role": "system",
    "content": "Assistant Instruction:\nUse short sentences. Prefer lists. Provide at most one example. End with a brief next step. If Arabic, write in Modern Standard Arabic."
  },
  {
    "role": "user",
    "content": "How do I connect Twitter to Lucidya?"
  }
]

----- Response -----
To connect Twitter to Lucidya, follow these steps:

1. Log in to your Lucidya account.
2. Go to the "Integrations" section.
3. Find Twitter in the list of available integrations.
4. Click on "Connect" or "Add."
5. Authorize Lucidya to access your Twitter account.
6. Follow any a

In [1]:
import os, time, json
from dotenv import load_dotenv
from openai import OpenAI

load_dotenv()
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# ---- Strategy A prompts ----
SYSTEM = (
    "You are a helpful Lucidya product assistant. Detect the user’s language and respond "
    "in the same language (Arabic ↔ English). Prioritize correctness, simplicity, and actionable steps. "
    "If unsure, ask one clarifying question before answering. Never invent features; if information is missing, "
    "say so and suggest what to check."
)

DEV = (
    "Assistant Instruction:\n"
    "Use short sentences. Prefer lists. Provide at most one example. "
    "End with a brief next step. If Arabic, write in Modern Standard Arabic."
)

def generate_response(end_user_query: str, kb_chunks: str | None = None) -> str:
    messages = [
        {"role": "system", "content": SYSTEM},
        {"role": "system", "content": DEV},
    ]
    if kb_chunks and kb_chunks.strip():
        messages.append({"role": "system", "content": "[Lucidya KB snippets]\n" + kb_chunks})

    messages.append({"role": "user", "content": end_user_query})

    # Log the full prompt
    print("----- Prompt sent to model -----")
    print(json.dumps(messages, indent=2, ensure_ascii=False))

    start = time.time()
    resp = client.chat.completions.create(
        model=os.getenv("OPENAI_MODEL", "gpt-4o-mini"),
        temperature=float(os.getenv("OPENAI_TEMPERATURE", "0.2")),
        messages=messages,
    )
    elapsed = time.time() - start

    text = resp.choices[0].message.content
    usage = resp.usage

    print("\n----- Response -----")
    print(text)
    print("\n----- Trace Info -----")
    print("Model:", resp.model)
    print("Prompt tokens:", usage.prompt_tokens)
    print("Completion tokens:", usage.completion_tokens)
    print("Total tokens:", usage.total_tokens)
    print("Elapsed time (s):", round(elapsed, 3))

    return text  # you can use it programmatically if needed

# ---- Example ----
if __name__ == "__main__":
    generate_response("How do I connect Twitter to Lucidya?")


----- Prompt sent to model -----
[
  {
    "role": "system",
    "content": "You are a helpful Lucidya product assistant. Detect the user’s language and respond in the same language (Arabic ↔ English). Prioritize correctness, simplicity, and actionable steps. If unsure, ask one clarifying question before answering. Never invent features; if information is missing, say so and suggest what to check."
  },
  {
    "role": "system",
    "content": "Assistant Instruction:\nUse short sentences. Prefer lists. Provide at most one example. End with a brief next step. If Arabic, write in Modern Standard Arabic."
  },
  {
    "role": "user",
    "content": "How do I connect Twitter to Lucidya?"
  }
]

----- Response -----
To connect Twitter to Lucidya, follow these steps:

1. Log in to your Lucidya account.
2. Go to the "Integrations" section.
3. Select "Twitter" from the list of available integrations.
4. Click on "Connect" or "Authorize."
5. Log in to your Twitter account if prompted.
6. Allow 